In [5]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


pd.set_option("display.max_columns", None)


engine = create_engine("postgresql+psycopg2://localhost:5430/retain_dev")

support_tickets = pd.read_sql(
    "SELECT * FROM support_tickets",
    engine
)

support_tickets.head()


,ticket_id,account_id,created_at,resolved_at,category,priority
0,TKT_D5107920,ACC_2D5C4D35,2025-05-11 17:40:00,2025-05-12 01:23:44.614746,Profile Issues,high
1,TKT_58E16F03,ACC_2D5C4D35,2024-05-12 12:00:00,2024-05-13 08:16:43.294669,App Crash,medium
2,TKT_1E171C29,ACC_2D5C4D35,2025-09-29 15:56:00,2025-10-01 08:47:46.166601,Buffering Issues,low
3,TKT_CDF3EBEE,ACC_2D5C4D35,2025-11-02 09:27:00,2025-11-03 11:52:56.007550,Feature Request,low
4,TKT_0437F558,ACC_893BC36A,2025-11-29 19:42:00,2025-11-30 22:07:01.928302,Subtitle Issues,high


In [28]:
#Ticket volume by category
support_tickets['category'].value_counts()


category
Cancellation Request     3793
Billing Dispute          3452
Payment Failure          3025
Video Quality            3023
Error Code               2998
Buffering Issues         2983
Streaming Lag            2977
App Crash                2969
Feature Request          2941
Playback Error           2937
Profile Issues           2926
Download Failed          2917
Login Problems           2909
Account Access           2891
Parental Controls        2887
Content Not Available    2874
Audio Sync               2865
Subtitle Issues          2855
Device Compatibility     2844
Black Screen             2803
Name: count, dtype: int64

In [6]:
#Resolution Time
support_tickets["created_at"] = pd.to_datetime(support_tickets["created_at"])
support_tickets["resolved_at"] = pd.to_datetime(support_tickets["resolved_at"])

support_tickets["resolution_hours"] = (
    support_tickets["resolved_at"] - support_tickets["created_at"]
).dt.total_seconds() / 3600

support_tickets["created_date"] = support_tickets["created_at"].dt.date
support_tickets["created_month"] = support_tickets["created_at"].dt.to_period("M").astype(str)


In [40]:
# When are tickets taking the longest to resolve?
support_tickets["weekday"] = support_tickets["created_at"].dt.day_name()
support_tickets["hour"] = support_tickets["created_at"].dt.hour

heatmap_data = (
    support_tickets
    .groupby(["weekday", "hour"])["resolution_hours"]
    .mean()
    .reset_index()
)

weekday_order = [
    "Monday", "Tuesday", "Wednesday",
    "Thursday", "Friday", "Saturday", "Sunday"
]

fig = px.density_heatmap(
    heatmap_data,
    x="hour",
    y="weekday",
    z="resolution_hours",
    category_orders={"weekday": weekday_order},
    color_continuous_scale="RdYlGn_r",
    title="Avg Resolution Time by Day & Hour"
)

fig.show()



In [29]:
#Avergae Resolution Time by priority
support_tickets.groupby('priority')['resolution_hours'].mean()


priority
high      18.257819
low       17.926793
medium    18.016502
Name: resolution_hours, dtype: float64

In [7]:
#KPI Snapshot
kpis = {
    "Total Tickets": len(support_tickets),
    "Avg Resolution (hrs)": round(support_tickets["resolution_hours"].mean(), 2),
    "Median Resolution (hrs)": round(support_tickets["resolution_hours"].median(), 2),
    "% High Priority": round(
        (support_tickets["priority"] == "high").mean() * 100, 1
    )
}

pd.DataFrame(kpis, index=["Value"])


,Total Tickets,Avg Resolution (hrs),Median Resolution (hrs),% High Priority
Value,59869,18.01,12.43,14.8


In [8]:
#Ticket Volume Over Time
tickets_over_time = (
    support_tickets
    .groupby("created_date")
    .size()
    .reset_index(name="tickets")
)

fig = px.line(
    tickets_over_time,
    x="created_date",
    y="tickets",
    title="Daily Support Ticket Volume",
    labels={"tickets": "Number of Tickets", "created_date": "Date"},
)

fig.show()


In [9]:
#what users complain about
category_counts = (
    support_tickets["category"]
    .value_counts()
    .reset_index()
)

category_counts.columns = ["category", "tickets"]

fig = px.bar(
    category_counts,
    x="category",
    y="tickets",
    title="Support Tickets by Category",
    text="tickets"
)

fig.update_traces(textposition="outside")
fig.show()


In [10]:
#Priority Distribution
fig = px.pie(
    support_tickets,
    names="priority",
    title="Ticket Priority Distribution",
    hole=0.45
)

fig.show()


In [11]:
#Resolution Time by Priority
fig = px.box(
    support_tickets,
    x="priority",
    y="resolution_hours",
    title="Resolution Time by Priority",
    labels={"resolution_hours": "Hours to Resolve"},
)

fig.show()


In [12]:
#Category × Priority Heatmap
heatmap_data = (
    support_tickets
    .groupby(["category", "priority"])
    .size()
    .reset_index(name="tickets")
)

fig = px.density_heatmap(
    heatmap_data,
    x="priority",
    y="category",
    z="tickets",
    color_continuous_scale="Blues",
    title="Ticket Volume Heatmap: Category vs Priority"
)

fig.show()


In [13]:
#Resolution Time Trend
resolution_trend = (
    support_tickets
    .groupby("created_month")["resolution_hours"]
    .mean()
    .reset_index()
)

fig = px.line(
    resolution_trend,
    x="created_month",
    y="resolution_hours",
    markers=True,
    title="Average Resolution Time Over Time",
    labels={"resolution_hours": "Avg Hours to Resolve"}
)

fig.show()


In [14]:
#Top Problem Accounts (account-level risk)
top_accounts = (
    support_tickets["account_id"]
    .value_counts()
    .head(10)
    .reset_index()
)

top_accounts.columns = ["account_id", "tickets"]

fig = px.bar(
    top_accounts,
    x="account_id",
    y="tickets",
    title="Top 10 Accounts by Support Tickets"
)

fig.show()


### Service Level Agreement actually means
An SLA is a promise the company makes about service quality.
In the context of support tickets, it usually means:
We promise to respond to or resolve this type of issue within X time.

In [15]:
#SLA & Support Performance Analysis
sla_hours = {"high": 4, "medium": 24, "low": 72}

support_tickets["sla_hours"] = support_tickets["priority"].map(sla_hours)
support_tickets["sla_breached"] = support_tickets["resolution_hours"] > support_tickets["sla_hours"]

support_tickets["sla_breached"].value_counts(normalize=True) * 100


sla_breached
False    82.391555
True     17.608445
Name: proportion, dtype: float64

82.4% of tickets met their SLA

17.6% of tickets breached their SLA

In other words:
~1 in every 6 support tickets is resolved too late

In [24]:
support_tickets.groupby("priority")["sla_breached"].mean() * 100


priority
high      64.004056
low        1.452110
medium    21.233797
Name: sla_breached, dtype: float64

64% of HIGH-priority tickets miss SLA

21% of MEDIUM-priority tickets miss SLA

1.5% of LOW-priority tickets miss SLA

The tickets that matter most are handled the worst.

In [26]:
support_tickets[support_tickets["priority"] == "high"] \
    .groupby("category")["sla_breached"] \
    .mean() \
    .sort_values(ascending=False)

#69.3% of HIGH-priority “Streaming Lag” tickets breached SLA

category
Streaming Lag            0.693023
Device Compatibility     0.691211
Video Quality            0.661836
Login Problems           0.654788
Account Access           0.654501
Download Failed          0.646081
App Crash                0.646018
Error Code               0.644309
Profile Issues           0.640000
Playback Error           0.635236
Black Screen             0.635036
Subtitle Issues          0.633094
Payment Failure          0.632967
Parental Controls        0.629977
Cancellation Request     0.626943
Buffering Issues         0.624079
Feature Request          0.623874
Audio Sync               0.622596
Billing Dispute          0.613508
Content Not Available    0.601432
Name: sla_breached, dtype: float64

In [27]:
#100% of this account’s HIGH-priority tickets breached SLA
support_tickets[support_tickets["priority"] == "high"] \
    .groupby("account_id")["sla_breached"] \
    .mean() \
    .sort_values(ascending=False) \
    .head(10)


account_id
ACC_00047CEB    1.0
ACC_94012930    1.0
ACC_93EEC953    1.0
ACC_93E82F88    1.0
ACC_93E056A5    1.0
ACC_93DE2033    1.0
ACC_93BA09AD    1.0
ACC_93B071B4    1.0
ACC_93A8A821    1.0
ACC_93A5F8BF    1.0
Name: sla_breached, dtype: float64

In [16]:
account_risk = (
    support_tickets
    .groupby("account_id")
    .agg(
        tickets=("ticket_id", "count"),
        avg_resolution=("resolution_hours", "mean"),
        high_priority_rate=("priority", lambda x: (x == "high").mean())
    )
    .reset_index()
)

account_risk.sort_values("tickets", ascending=False).head()


,account_id,tickets,avg_resolution,high_priority_rate
26900,ACC_FF8698B4,10,15.906560,0.000000
13215,ACC_7CCD65C5,10,14.267464,0.100000
14223,ACC_867D9316,9,11.029718,0.000000
7224,ACC_43C33CBC,9,19.809286,0.000000
15461,ACC_922F4B42,9,20.767926,0.444444


In [30]:
account_risk["risk_score"] = (
    account_risk["tickets"] * 0.4 +
    account_risk["avg_resolution"] * 0.4 +
    account_risk["high_priority_rate"] * 100 * 0.2
)

account_risk.sort_values("risk_score", ascending=False).head(10)


,account_id,tickets,avg_resolution,high_priority_rate,risk_score
9738,ACC_5B9332A1,1,168.000000,1.0,87.600000
8403,ACC_4F0521E3,1,132.687922,1.0,73.475169
6736,ACC_3F0678F0,1,119.288511,1.0,68.115404
24828,ACC_EB5FC975,2,168.000000,0.0,68.000000
23861,ACC_E1EFB3A4,1,117.986163,1.0,67.594465
13938,ACC_838FD34E,1,116.517027,1.0,67.006811
26453,ACC_FB47DD23,1,104.307622,1.0,62.123049
5239,ACC_30E2EB35,1,104.279880,1.0,62.111952
22959,ACC_D9839932,2,100.510166,1.0,61.004066
17228,ACC_A38D0091,2,149.345824,0.0,60.538330


In [17]:
#Avg resilution hours by category - helps with skill staffing
support_tickets.groupby("category")["resolution_hours"].mean().sort_values(ascending=False)


category
Device Compatibility     18.896096
Feature Request          18.594270
Error Code               18.495481
Account Access           18.472286
Login Problems           18.308717
App Crash                18.295744
Subtitle Issues          18.270812
Audio Sync               18.269550
Billing Dispute          18.106878
Streaming Lag            18.037072
Black Screen             18.010721
Download Failed          17.918485
Payment Failure          17.908915
Video Quality            17.888263
Parental Controls        17.795091
Buffering Issues         17.568613
Cancellation Request     17.535215
Content Not Available    17.395184
Playback Error           17.318752
Profile Issues           17.202956
Name: resolution_hours, dtype: float64

In [18]:
category_impact = (
    support_tickets
    .groupby("category")
    .agg(
        tickets=("ticket_id", "count"),
        avg_resolution=("resolution_hours", "mean")
    )
    .reset_index()
)

category_impact


,category,tickets,avg_resolution
0,Account Access,2891,18.472286
1,App Crash,2969,18.295744
2,Audio Sync,2865,18.269550
3,Billing Dispute,3452,18.106878
4,Black Screen,2803,18.010721
5,Buffering Issues,2983,17.568613
6,Cancellation Request,3793,17.535215
7,Content Not Available,2874,17.395184
8,Device Compatibility,2844,18.896096
9,Download Failed,2917,17.918485


In [32]:
#ticket creation understanding

import pandas as pd
import plotly.express as px

# Ensure datetime
support_tickets["created_at"] = pd.to_datetime(support_tickets["created_at"])

# Extract time features
support_tickets["hour"] = support_tickets["created_at"].dt.hour
support_tickets["weekday"] = support_tickets["created_at"].dt.day_name()




In [33]:
# Tickets per day per hour
daily_counts = (
    support_tickets
    .groupby([
        support_tickets["created_at"].dt.date,
        "weekday",
        "hour"
    ])
    .size()
    .reset_index(name="tickets")
)

# Average tickets per weekday/hour
avg_heatmap = (
    daily_counts
    .groupby(["weekday", "hour"])["tickets"]
    .mean()
    .reset_index()
)


In [34]:
weekday_order = [
    "Monday", "Tuesday", "Wednesday",
    "Thursday", "Friday", "Saturday", "Sunday"
]

avg_heatmap["weekday"] = pd.Categorical(
    avg_heatmap["weekday"],
    categories=weekday_order,
    ordered=True
)


In [35]:
fig = px.density_heatmap(
    avg_heatmap,
    x="hour",
    y="weekday",
    z="tickets",
    color_continuous_scale="Blues",
    title="Average Support Tickets by Hour & Day of Week",
    labels={
        "hour": "Hour of Day",
        "weekday": "Day of Week",
        "tickets": "Avg Tickets"
    }
)

fig.update_layout(
    height=450,
    xaxis=dict(tickmode="linear"),
)

fig.show()


In [36]:
avg_heatmap_high = avg_heatmap.merge(
    support_tickets[["weekday", "hour", "priority"]],
    on=["weekday", "hour"]
)


In [37]:
import streamlit as st
st.plotly_chart(fig, use_container_width=True)


2026-01-23 20:29:35.107 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-01-23 20:29:35.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 20:29:35.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 20:29:35.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 20:29:35.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 20:29:35.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [20]:
#pain_score = resolution_hours × priority_weight

support_tickets["pain_score"] = (
    support_tickets["resolution_hours"] * 
    support_tickets["priority"].map({"high": 3, "medium": 2, "low": 1})
)

cx_score = (
    support_tickets
    .groupby("account_id")["pain_score"]
    .sum()
    .reset_index()
    .sort_values("pain_score", ascending=False)
)

cx_score.head()


,account_id,pain_score
18822,ACC_B24089A0,535.843418
3050,ACC_1CD3574E,526.519350
9311,ACC_574DAE01,520.716227
16916,ACC_A0458A30,517.101959
16000,ACC_9707D9D5,506.843971


In [23]:
# app.py
import streamlit as st
st.title("Support Ticket Dashboard")


2026-01-23 16:50:17.511 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 16:50:17.602 
  command:

    streamlit run /Users/tej/Desktop/CAPSTONE/retain_project/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-01-23 16:50:17.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-23 16:50:17.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()